## IMPORTANTE !

La estructura de las carpetas para correr este jupyter notebook debe ser la siguiente. Debe crearse una carpeta llamada "Datos Comerciales" dentro de la cual habrá una carpeta para las "Exportaciones", en esta existirán carpetas para todos los años disponibles en las cuales los archivos descargados del [repositorio de datos de aduanas](https://datos.gob.cl/organization/servicio_nacional_de_aduanas) deberán abrirse en MS Excel y ser guardados con el nombre del mes en formato .csv. Cualquier cambio a esta forma de estructurar los directorios deberá ser concordante y reflejado en las funciones `create_region_csvs()` y `create_chile_csvs()`.
<br>
<img src="./images/Carpetas.png">
<br>

<img src="./images/Carpetas2.png">

# ¿Qué es el RCA?
Se han usado medidas de ventaja comparativa revelada (RCA) para ayudar a evaluar el potencial de exportación de un país. La RCA indica si un país está en proceso de ampliar los productos en los que tiene potencial comercial, en contraposición a situaciones en las que el número de productos que se pueden exportar de manera competitiva es estático. También puede aportar información útil acerca de posibles perspectivas comerciales con nuevos asociados.En este caso no se analizará a nivel de paises sino de regiones.


<br>
$ RCAe =  ln\left [ \left ( \frac{X_{ij}}{X_{it}} \right )/\left ( \frac{X_{im}}{X_{tm}} \right ) \right ] $

Donde:
* $X_{ij}$ - Corresponde a las exportaciones del producto $i$ en la región $j$
* $X_{tj}$ - Corresponde a las exportaciones totales en la región $j$
* $X_{im}$ - Corresponde a las exportaciones del producto $i$ en la región $j$
* $X_{tm}$ - Corresponde a las exportaciones totales del mundo

<br>

El contenido de este documento busca calcular el RCA de los prodcutos exportados por la Región de los Ríos por lo que la formula queda de la siguiente forma.
<br>
$ RCAe =  ln\left [ \left ( \frac{X_{ij}}{X_{it}} \right )/\left ( \frac{X_{ip}}{X_{tp}} \right ) \right ] $

Donde:

* $X_{ij}$ - Corresponde a las exportaciones del producto $i$ en Los Ríos
* $X_{tj}$ - Corresponde a las exportaciones totales en Los Ríos
* $X_{ip}$ - Corresponde a las exportaciones del producto $i$ en Chile
* $X_{tp}$ - Corresponde a las exportaciones totales de Chile


## Definición de funciones que ayudarán al analisis
***
- `create_region_csvs()` : Función que se mueve por los directorios dentro de la carpeta "Exportaciones" filtrando los datos de aduanas solo dejando las columnas relevantes para los registros en donde la comuna reportada por el exportador comuna a una perteneciente a la región de Los Ríos (Códigos entre 14000 y 15000), creando para cada archivo un archivo aparte de la forma {nombre_mes}_LR.csv.
***
- `create_chile_csvs()` : Función que se mueve por los directorios dentro de la carpeta "Exportaciones" filtrando los datos de aduanas solo dejando las columnas relevantes para los registros en donde la comuna reportada por el exportador comuna a una perteneciente a la región de Los Ríos (Códigos entre 14000 y 15000), creando para cada archivo un archivo aparte de la forma {nombre_mes}_LR.csv.
***
- `unify_df` : Función que tiene por entrada el código de archivos a unificar, los cuales en este caso podrán ser "LR" o "CL" correspondientes a los codigos distintivos de los archivos creados por `create_region_csvs()` y `create_chile_csvs()` , esta función unifica los registros de todos los años y meses en un solo archivo llamado exp_{distintivo}.csv
***
- `prepare_df` : Función que tiene por entrada un dataframe generado por alguno de los archivos resultantes de la función `unify_df`, los cuales son preprocesados cambiando tipos de variables y nombres de columnas.
***
- `create_df` : Función que tiene por entrada el dataframe gnerado por `prepare_df` el cual es reagrupado para poder ser input de la función `filter_df`.
***
- `filter_df` : Función que filtra en el archivo de las exportaciones nacionales todos los códigos arancelarios exportados por Los Ríos para los años correspondientes de su exportación, agrupando las exportaciones anuales de cada articulo y realizando una sumatoria de sus valores en FOB.
***
- `final_df` : Funcion que calcula los totales exportados por la región y el país para cada año para poder aplicar la formula del RCAe y asignar los valores en una columna. Posterior a este calculo procede a agrupar los Códigos Arancelarios por Sección arancelaria la cual está disponible en [el sitio web de Aduanas](https://www.aduana.cl/arancel-aduanero-vigente/aduana/2016-12-30/090118.html).
***
- `rca_frecuency` : Tiene como inputs los dataframes que contienen la suma de articulos con RCAe deficiente y otro con la suma de articulos con RCAe competitivo por año y sección arancelaria,
***

## Lógica:

<img src="./images/logica.png" width="1200">

# Análisis y calculos de RCAe para cada año y articulo exportado desde Los Ríos.

In [1]:
import pandas as pd
import math

In [2]:
meses = ["Enero", "Febrero", "Marzo", "Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]
años = {
    2010:meses,
    2011:meses,
    2012:meses,
    2013:meses,
    2014:meses,
    2015:meses,
    2016:meses,
    2017:meses,
    2018:meses,
    2019:meses,
    2020:meses,
    2021:meses
}

In [3]:
def create_region_csvs(codigos_region,region):
    for key in años:
        for mes in años[key]:
            base_path = 'Datos comerciales/Exportaciones'
            initial_path = '{}/{}/{}.csv'.format(base_path,key,mes)
            df = pd.read_csv(initial_path,low_memory=False)
            columns = ['Column1','Column8','Column70','Column74']
            df = df.loc[:, columns]
            mask = (df['Column8']>=codigos_region[0])&(df['Column8']<=codigos_region[1])
            df = df[mask]
            final_path = '{}/{}/{}_{}.csv'.format(base_path,key,mes,region)
            df["Column1"] = str(key)
            df.rename({"Column1": "FECHA", "Column8": "COMUNA","Column70": "CODIGO_ARANCEL","Column74": "FOB_US"},axis='columns')
            df.to_csv(final_path,encoding='utf-8')

In [4]:
def create_chile_csvs():
    for key in años:
        for mes in años[key]:
            base_path = 'Datos comerciales/Exportaciones'
            initial_path = '{}/{}/{}.csv'.format(base_path,key,mes)
            df = pd.read_csv(initial_path,low_memory=False)
            columns = ['Column1','Column8','Column70','Column74']
            df = df.loc[:, columns]
            final_path = '{}/{}/{}_CL.csv'.format(base_path,key,mes)
            df["Column1"] = str(key)
            df.rename({"Column1": "FECHA", "Column8": "COMUNA","Column70": "CODIGO_ARANCEL","Column74": "FOB_US"},axis='columns')
            df.to_csv(final_path,encoding='utf-8')


In [5]:
def unify_dfs(identificador):
    frames = []
    for key in años:
        for mes in años[key]:
            base_path = 'Datos comerciales/Exportaciones'
            initial_path = '{}/{}/{}_{}.csv'.format(base_path,key,mes,identificador)
            df = pd.read_csv(initial_path)
            frames.append(df)

    df_final= pd.concat(frames)
    df_final.to_csv("exp_{}.csv".format(identificador),index=False)

In [6]:
def prepare_df(df):
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    # En caso de que las columnas con numeros que representan codigos hayan sido leido como numeros con decimales
    # Estos son transformados y preparados para su correcto uso
    df["Column8"] = df["Column8"].astype(str)
    df["Column8"] = df["Column8"].apply(lambda row: row.split('.')[0])
    df["Column70"] = df["Column70"].astype(str)
    df["Column70"] = df["Column70"].apply(lambda row: row.split('.')[0])
    df["Column74"] = df["Column74"].astype(str)
    df["Column74"] = df["Column74"].apply(lambda row: row.replace(',', '.'))
    df["Column74"] = df["Column74"].astype(float)
    # Se renombran las columnas según los codigos que _lrentrega Aduanas
    df.rename(columns={'Column1': 'Fecha', 'Column8': 'Cod_Comuna', 'Column70': 'Cod_Arancelario', 'Column74': 'FOB_USD'},inplace=True)
    df_group = df.groupby(['Fecha', 'Cod_Arancelario']).agg({'FOB_USD': 'sum'})
    return df_group

In [7]:
def create_df(df_suma):
    d={}
    i=0
    for index, row in df_suma.iterrows():
        d[i] = [index[0],index[1],int(row)]
        i += 1
    df = pd.DataFrame.from_dict(d, orient='index',columns=["Año Comercial","Codigo Arancelario","FOB_USD"])
    return df

In [8]:
def filter_df(df,dict):
    dfs_list = []
    for key in dict:
        df_year_filtered = df[df["Fecha"]==key]
        df_codes_filtered = df_year_filtered[df_year_filtered["Cod_Arancelario"].isin(dict[key])]
        dfs_list.append(df_codes_filtered)
    df_final = pd.concat(dfs_list)
    df_final = df_final.drop(['Cod_Comuna'], axis=1)
    df_final_group = df_final.groupby(['Fecha','Cod_Arancelario']).agg({'FOB_USD': 'sum'})
    return df_final_group

In [9]:
def final_df(merged_filtered_df):

    for index, value in merged_filtered_df.iterrows():
        # Se calcula el total en FOB de exportaciones de la región para el año de cada registro
        total_exp_region = merged_filtered_df.loc[merged_filtered_df['Año Comercial'] == value["Año Comercial"], 'FOB_USD_LR'].sum()
        # Se calcula el total en FOB de exportaciones  del pais para el año de cada registro
        total_exp_pais = merged_filtered_df.loc[merged_filtered_df['Año Comercial'] == value["Año Comercial"], 'FOB_USD_CL'].sum()
        # Se calculan las divisiones de la formula del RCAe
        pre_RCAe = float((value["FOB_USD_LR"]/total_exp_region)/(value["FOB_USD_CL"]/total_exp_pais))
        # Si el valor es mayor a 0 se le aplica el log al valor, asignando el resultado en una columna definida como RCAe
        if pre_RCAe>0:
            merged_filtered_df.loc[index,"RCAe"] = math.log(pre_RCAe)
        # De manera contraria se asigna a la columna RCAe el valor de 0, no se aplica el log debido a su indefinición en 0
        else:
            merged_filtered_df.loc[index,"RCAe"] = 0

    # Se divide en 1M el código arancelario dejando de esta manera los primeros 2 digitos con los que luego se pueden agrupar en secciones
    merged_filtered_df["Codigo Arancelario"] = merged_filtered_df["Codigo Arancelario"].astype(float)
    merged_filtered_df["GRUPO"] = merged_filtered_df["Codigo Arancelario"]/1000000
    merged_filtered_df["Codigo Arancelario"] = merged_filtered_df["Codigo Arancelario"].astype(str)

    # Se redonde ael valor de la división anterior
    for index, value in merged_filtered_df.iterrows():
        merged_filtered_df.loc[index,"GRUPO"] = str(math.floor(value["GRUPO"]))

    # Se clasifican los resultados según la sección arancelaria a la que correspondan.
    for index, value in merged_filtered_df.iterrows():
        valor = math.floor(float(value["GRUPO"]))
        if valor>=0 and valor<=5:
            merged_filtered_df.loc[index,"Seccion"] = "1"
        if valor>=6 and valor<=14:
            merged_filtered_df.loc[index,"Seccion"] = "2"
        if valor==15:
            merged_filtered_df.loc[index,"Seccion"] = "3"
        if valor>=16 and valor<=24:
            merged_filtered_df.loc[index,"Seccion"] = "4"
        if valor>=25 and valor<=27:
            merged_filtered_df.loc[index,"Seccion"] = "5"
        if valor>=28 and valor<=38:
            merged_filtered_df.loc[index,"Seccion"] = "6"
        if valor>=39 and valor<=40:
            merged_filtered_df.loc[index,"Seccion"] = "7"
        if valor>=41 and valor<=43:
            merged_filtered_df.loc[index,"Seccion"] = "8"
        if valor>=44 and valor<=46:
            merged_filtered_df.loc[index,"Seccion"] = "9"
        if valor>=47 and valor<=49:
            merged_filtered_df.loc[index,"Seccion"] = "10"
        if valor>=50 and valor<=63:
            merged_filtered_df.loc[index,"Seccion"] = "11"
        if valor>=64 and valor<=67:
            merged_filtered_df.loc[index,"Seccion"] = "12"
        if valor>=68 and valor<=70:
            merged_filtered_df.loc[index,"Seccion"] = "13"
        if valor==71:
            merged_filtered_df.loc[index,"Seccion"] = "14"
        if valor>=72 and valor<=83:
            merged_filtered_df.loc[index,"Seccion"] = "15"
        if valor>=84 and valor<=85:
            merged_filtered_df.loc[index,"Seccion"] = "16"
        if valor>=86and valor<=89:
            merged_filtered_df.loc[index,"Seccion"] = "17"
        if valor>=90 and valor<=92:
            merged_filtered_df.loc[index,"Seccion"] = "18"
        if valor==93:
            merged_filtered_df.loc[index,"Seccion"] = "19"
        if valor>=94 and valor<=96:
            merged_filtered_df.loc[index,"Seccion"] = "20"
        if valor>=97 and valor<=99:
            merged_filtered_df.loc[index,"Seccion"] = "21"

    # Se elimina la columna generada anteriormente
    merged_filtered_df=merged_filtered_df.drop(columns=['GRUPO'])
    # Se redondean los valores del RCAe a un solo decimal
    merged_filtered_df["RCAe"] = merged_filtered_df["RCAe"].round(1)
    merged_filtered_df["Codigo Arancelario"] = merged_filtered_df["Codigo Arancelario"].str[:-2]
    return merged_filtered_df

In [10]:
def rca_frecuency(df_bad_rca,df_good_rca):
    d={}
    año=[]
    seccion=[]
    bad_rca_count=[]

    for i in range(0,len(df_bad_rca.index)):
        año.append(df_bad_rca.index[i][0])
        seccion.append(df_bad_rca.index[i][1])
        bad_rca_count.append(df_bad_rca.iloc[i])

    d["Año Comercial"] = año
    d["Seccion"] = seccion
    d["RCA < 1"] = bad_rca_count
    df_bad = pd.DataFrame.from_dict(d)

    d={}
    año=[]
    seccion=[]
    good_rca_count=[]

    for i in range(0,len(df_good_rca.index)):
        año.append(df_good_rca.index[i][0])
        seccion.append(df_good_rca.index[i][1])
        good_rca_count.append(df_good_rca.iloc[i])

    d["Año Comercial"] = año
    d["Seccion"] = seccion
    d["RCA > 1"] = good_rca_count
    df_good = pd.DataFrame.from_dict(d)
    df_good = df_good.drop(columns=['Año Comercial','Seccion'])
    frecuency_rca = pd.concat([df_bad,df_good], axis=1)

    return frecuency_rca

In [11]:
create_region_csvs([14000,15000],'LR')
create_chile_csvs()

In [12]:
unify_dfs('LR')
unify_dfs('CL')

In [13]:
#-----------------LOS RIOS----------------------#
# Se leen los CSVS unificados de exportaciones de Los Ríos
df_exportaciones_lr = pd.read_csv("exp_lr.csv")
# Se cambian los nombres de columnas y se genera un dataframe groupby
df_exportaciones_lr_group = prepare_df(df_exportaciones_lr)
# Se crea el dataframe que muestra por año el total en FOB (USD) de exportaciones para c/código registrado como exportado
df_final_lr = create_df(df_exportaciones_lr_group)


#-----------------CHILE------------------------#
# Se leen los CSVS unificados de exportaciones de Chile
df_exportaciones_cl = pd.read_csv("exp_cl.csv")
# Se cambian los nombres de columnas y se genera un dataframe groupby
df_exportaciones_cl_group = prepare_df(df_exportaciones_cl)
# Se crea el dataframe que muestra por año el total en FOB (USD) de exportaciones para c/código registrado como exportado
df_final_cl = create_df(df_exportaciones_cl_group)

c:\users\manue\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
df_exportaciones_lr

,Fecha,Cod_Comuna,Cod_Arancelario,FOB_USD
0,2010,14201,4041000,29125.0
1,2010,14101,3026812,789706.0
2,2010,14201,4041000,22680.0
3,2010,14201,4069010,8345796.0
4,2010,14201,4069010,833601.0
...,...,...,...,...
19000,2021,14101,6011011,5429705.0
19001,2021,14101,6011011,3067612.0
19002,2021,14201,4022118,367500.0
19003,2021,14101,6011011,3374153.0


In [15]:
# Se crea un diccionario que contenga por año cada tipo de producto exportado desde Los Ríos por su código arancelario para ser usado como filtro posteriormente

years_dict={
    2010:list(df_final_lr[df_final_lr["Año Comercial"]==2010]["Codigo Arancelario"]),
    2011:list(df_final_lr[df_final_lr["Año Comercial"]==2011]["Codigo Arancelario"]),
    2012:list(df_final_lr[df_final_lr["Año Comercial"]==2012]["Codigo Arancelario"]),
    2013:list(df_final_lr[df_final_lr["Año Comercial"]==2013]["Codigo Arancelario"]),
    2014:list(df_final_lr[df_final_lr["Año Comercial"]==2014]["Codigo Arancelario"]),
    2015:list(df_final_lr[df_final_lr["Año Comercial"]==2015]["Codigo Arancelario"]),
    2016:list(df_final_lr[df_final_lr["Año Comercial"]==2016]["Codigo Arancelario"]),
    2017:list(df_final_lr[df_final_lr["Año Comercial"]==2017]["Codigo Arancelario"]),
    2018:list(df_final_lr[df_final_lr["Año Comercial"]==2018]["Codigo Arancelario"]),
    2019:list(df_final_lr[df_final_lr["Año Comercial"]==2019]["Codigo Arancelario"]),
    2020:list(df_final_lr[df_final_lr["Año Comercial"]==2020]["Codigo Arancelario"]),
    2021:list(df_final_lr[df_final_lr["Año Comercial"]==2021]["Codigo Arancelario"]),
}

In [16]:
df_exportaciones_cl_filtered = filter_df(df_exportaciones_cl,years_dict)
df_exportaciones_cl_filtered

FOB_USD
Fecha Cod_Arancelario              
2010  1019011          2.302353e+06
      16059099         5.701207e+08
      19019011         2.407464e+08
      20089990         6.306142e+07
      20098020         5.695079e+08
...                             ...
2021  85444200         7.023149e+07
      90158000         1.272206e+08
      90178000         3.935102e+06
      90303100         8.640472e+06
      96200000         5.959130e+05

[935 rows x 1 columns]

In [17]:
df_final_cl = create_df(df_exportaciones_cl_filtered)
# Eliminación de columnas de año comercial y codigo arancelario para que no se repitan una vez se junten los dataframes de Los Rïos y Chile.
df_final_cl = df_final_cl.drop(columns=['Año Comercial','Codigo Arancelario'])
# Se renombran las columnas
df_final_cl = df_final_cl.rename(columns={"FOB_USD": "FOB_USD_CL"})
df_final_lr = df_final_lr.rename(columns={"FOB_USD": "FOB_USD_LR"})
# Concatenación de daframes de Los Ríos y Nacional
merged_dfs = pd.concat([df_final_lr, df_final_cl], axis=1, join="inner")

In [18]:
final_df = final_df(merged_dfs)
final_df

,Año Comercial,Codigo Arancelario,FOB_USD_LR,FOB_USD_CL,RCAe,Seccion
0,2010,1019011,20000,2302353,-0.8,1
1,2010,16059099,16570500,570120664,0.4,4
2,2010,19019011,570632,240746425,-2.1,4
3,2010,20089990,22339781,63061421,2.9,4
4,2010,20098020,149998748,569507905,2.6,4
...,...,...,...,...,...,...
930,2021,85444200,38237,70231486,-3.9,16
931,2021,90158000,2739941,127220633,-0.2,18
932,2021,90178000,2362055,3935102,3.2,18
933,2021,90303100,23914,8640472,-2.2,18


In [19]:
# Se agrupa el df llamado final_df según Año comercial y Sección arancelaria
df_rca_conditional = final_df.groupby(['Año Comercial','Seccion'])
# A este df agrupado se le cuentan por grupo (año comercial y sección arancelaria) cuantos articulos por sección tuvieron un RCAe < 1 (poco competitivo)
df_bad_rca = df_rca_conditional.apply(lambda x: x[x['RCAe'] < 1 ]['Seccion'].count())
# A este df agrupado se le cuentan por grupo (año comercial y sección arancelaria) cuantos articulos por sección tuvieron un RCAe > 1 (competitivo)
df_good_rca = df_rca_conditional.apply(lambda x: x[x['RCAe'] > 1 ]['Seccion'].count())

In [20]:
df_rca_frecuency = rca_frecuency(df_bad_rca,df_good_rca)

In [21]:
df_rca_frecuency.Seccion = df_rca_frecuency.Seccion.astype(float)

In [22]:
df_rca_frecuency.sort_values(by=['Año Comercial','Seccion'])

,Año Comercial,Seccion,RCA < 1,RCA > 1
0,2010,1.0,12,6
5,2010,2.0,10,2
7,2010,4.0,3,5
8,2010,6.0,3,0
9,2010,8.0,1,0
...,...,...,...,...
142,2021,13.0,1,0
143,2021,15.0,6,0
144,2021,16.0,19,1
145,2021,18.0,2,1


In [23]:
# Creación de CSV
df_rca_frecuency.to_csv('rca_frecuency.csv')
# Creación de CSV
final_df.to_csv('final_df.csv')
final_df["Año Comercial"] = final_df["Año Comercial"].astype(str)

In [24]:
final_df

,Año Comercial,Codigo Arancelario,FOB_USD_LR,FOB_USD_CL,RCAe,Seccion
0,2010,1019011,20000,2302353,-0.8,1
1,2010,16059099,16570500,570120664,0.4,4
2,2010,19019011,570632,240746425,-2.1,4
3,2010,20089990,22339781,63061421,2.9,4
4,2010,20098020,149998748,569507905,2.6,4
...,...,...,...,...,...,...
930,2021,85444200,38237,70231486,-3.9,16
931,2021,90158000,2739941,127220633,-0.2,18
932,2021,90178000,2362055,3935102,3.2,18
933,2021,90303100,23914,8640472,-2.2,18


In [25]:
# Creación de JSON el cual será desplegado en el DSS luego de ser procesado para ello con un script de JavaScript
json_output = final_df.to_json(path_or_buf="json_RCA.json",orient="records")

## Conteo de cantidad de articulos exportados por sección y año con RCAe competitivos y no competitivos

In [26]:
years = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]
sectors = ["S1","S2","S3","S4","S5","S6","S7","S8","S9","S10","S11","S12","S13","S14","S15","S16","S17","S18","S19","S20","S21"]
d={}
l=[]
for year in years:
    l=[]
    for i in range(0,21):
        cond1 = final_df["Año Comercial"] == str(year)
        cond2 = final_df["Seccion"]==str(i+1)
        temp = final_df[(cond1) & (cond2)]
        cant=temp["Codigo Arancelario"].value_counts().sum()
        l.append(cant)
    d[year]=l

In [27]:
conteo = pd.DataFrame.from_dict(d, orient='index')
conteo = pd.DataFrame.from_dict(d, orient='index',columns=sectors)
#Creación de CSV
conteo.to_csv("conteo.csv")

In [28]:
conteo

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S12,S13,S14,S15,S16,S17,S18,S19,S20,S21
2010,19,12,0,8,0,3,0,1,17,0,...,0,0,0,1,7,6,1,0,1,0
2011,25,15,0,10,0,1,1,3,11,0,...,0,0,0,0,8,1,0,0,0,0
2012,23,8,0,15,0,0,1,1,6,4,...,0,2,0,2,8,3,1,0,0,0
2013,25,5,0,12,0,1,0,2,5,0,...,0,1,0,1,13,5,1,0,2,0
2014,24,6,0,12,0,0,3,2,4,1,...,0,1,0,4,18,6,0,0,1,0
2015,23,10,0,12,0,1,1,2,5,0,...,0,1,0,0,11,3,0,0,1,0
2016,19,7,0,13,1,1,4,2,6,2,...,0,2,0,5,12,4,2,0,0,0
2017,25,13,0,13,0,2,2,3,5,1,...,0,1,0,2,15,1,4,0,1,0
2018,23,11,0,16,1,1,3,2,5,1,...,1,2,0,4,8,3,1,0,1,0
2019,13,10,0,18,0,1,7,1,5,2,...,1,1,0,2,13,1,4,0,2,0
